## Football Matches Recommendation System

### Objective:
To have a system that recommends football games/matches based on a user's prefrence

### Using the system:
The user will have to choose a specific date and then the games on that date will be presented to him and he has to choose one of them as a prefered one and then the system will start recommending

### Assumptions:
All of the upcoming games characteristics are euqlals to the mean of the characteristics of the last 10 seasons.

##  The System

### Importing Packages

In [338]:
import glob
import os
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn import neighbors
from sklearn.preprocessing import MinMaxScaler

### Importing All Games Data set

In [339]:
All_games = pd.read_csv('Data/All_games.csv')
All_games.drop('Unnamed: 0', axis= 1, inplace= True)
All_games.sample(5)

,Game,Attendance,Total_goals,goals_diff,Game_Class,League
642,Amiens vs Rennes,10319.500000,2,1,Closed_Game,FL
2261,Estoril vs Vitória,1463.800000,1,1,Closed_Game,P
664,Angers vs Nantes,12479.166667,2,2,Competitive_Game,FL
1101,Benevento vs Torino,11416.000000,2,0,Closed_Game,IT
2048,Dortmund vs Stuttgart,76296.800000,4,2,Competitive_Open_Game,BL


### Importing Upcoming Game Dataset

In [340]:
#read upcoming games dataset
Upcomig_Games = pd.read_csv('Data/Upcomig_Games.csv')

In [341]:
# This function is used to return a list of the games on a specific day
Upcomig_Games['Month'] = Upcomig_Games['Month'].astype(str)
Upcomig_Games['Day_n'] = Upcomig_Games['Day_n'].astype(str)
def game_date(month , day):
    games_list=[]
    for i in range(len(Upcomig_Games)):
        if Upcomig_Games['Month'][i] == month and Upcomig_Games['Day_n'][i] == day:
                games_list.append(Upcomig_Games['Game'][i])                 
    return games_list

### Inputting the Date

In [342]:
# This function is used to return a dataframe containing only the games on the previous list (Games on a specified date)
All_games2 = pd.DataFrame()
def inputgame(d):
    global All_games2
    for i in range(len(d)):
        All_games2=  All_games2.append(All_games[All_games['Game'].str.contains(d[i])])
    return All_games2.reset_index(drop= True, inplace=True)

In [343]:
#calling the function for the games on the 13th of march
games_list = game_date('3' , '13')

In [344]:
inputgame(games_list)
pd.set_option('display.max_rows', All_games2.shape[0]+1)
All_games2

,Game,Attendance,Total_goals,goals_diff,Game_Class,League
0,Granada vs Elche,15230.000000,1,1,Closed_Game,SP
1,Real Sociedad vs Alavés,24205.000000,2,2,Competitive_Game,SP
2,Getafe vs Valencia,10639.600000,2,1,Closed_Game,SP
3,Levante vs Espanyol,17241.400000,3,1,Competitive_Game,SP
4,Betis vs Athletic Club,40053.000000,2,1,Closed_Game,SP
5,Atlético Madrid vs Cádiz,NaN,4,4,One_Side_Game,SP
6,Rayo Vallecano vs Sevilla,11868.333333,2,1,Closed_Game,SP
7,Villarreal vs Celta Vigo,16324.000000,4,2,Competitive_Open_Game,SP
8,Barcelona vs Osasuna,63989.000000,6,5,One_Side_Game,SP
9,Mallorca vs Real Madrid,20275.000000,3,2,Competitive_Game,SP


### Preperations

In [345]:
# Encoding categorical data and preparing numarical data
Game = pd.get_dummies(All_games2['Game'])
Game_Class = pd.get_dummies(All_games2['Game_Class'])
League=pd.get_dummies(All_games2['League'])
Total_goals = All_games2['Total_goals']
goals_diff = All_games2['goals_diff']


In [346]:
#Scalling the attendance feaqture
Attendance = All_games2['Attendance']
Attendance = pd.DataFrame(Attendance.fillna(All_games2.Attendance.mean()))
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
Attendance= pd.DataFrame(sc.fit_transform(Attendance))
Attendance =Attendance[0]

In [347]:
#Concating the data before the modeling preocess
features = pd.concat([Total_goals, goals_diff, League, Attendance,
                      Game_Class], axis=1)

### Model

In [348]:
#Using KNN
model = neighbors.NearestNeighbors(n_neighbors=len(All_games2), algorithm='ball_tree')
model.fit(features)
dist, idlist = model.kneighbors(features)

In [349]:
#Calculating the cosine_similarity
cosSimilarities = cosine_similarity(features)
# store the results into a pandas dataframe
cos_similarities_df = pd.DataFrame(cosSimilarities)

### Function to Generate an Output

In [350]:
# This fincion is used to return the recomendations based on the lowest distance from the input
def gameRecommender(game_name):
    game_list_name = []
    game_list_distance = []
    game_list_similarity = []
    game_id = All_games2[All_games2['Game'] == game_name].index
    game_id = game_id[0]
    
    similarity = cos_similarities_df

    for newid in idlist[game_id]:
        game_list_name.append(All_games2.loc[newid].Game)
    
    for i in range(len(idlist)):   
        game_list_distance.append(dist[game_id][i])

    for newid in idlist[game_id]:   
        game_list_similarity.append(similarity[game_id][newid])

    print("-----------------------------------------------------------------------")
    print("Favorite Football Game:", game_name)
    
    print('The Recommendation order is built based on lowest distance')
    print('Similarity is just shown to be useful when the distances are equals (Can be ignored)')
    for i in range(7):
        if game_name != game_list_name[i]:    
            print("-----------------------------------------------------------------------")
            print(game_list_name[i])
            print('Distance =' , game_list_distance[i])
            print('Similarity =', game_list_similarity[i])
    return

### Inputting the Prefered Game

In [351]:
game_names = gameRecommender('Arsenal vs Leicester City')
game_names

-----------------------------------------------------------------------
Favorite Football Game: Arsenal vs Leicester City
The Recommendation order is built based on lowest distance
Similarity is just shown to be useful when the distances are equals (Can be ignored)
-----------------------------------------------------------------------
Paris S-G vs Bordeaux
Distance = 1.7439411895772692
Similarity = 0.921810641175396
-----------------------------------------------------------------------
Chelsea vs Newcastle Utd
Distance = 2.0053613995930157
Similarity = 0.902666861659122
-----------------------------------------------------------------------
Betis vs Athletic Club
Distance = 2.695765697179594
Similarity = 0.8111159177841787
-----------------------------------------------------------------------
Milan vs Empoli
Distance = 2.8211458916472028
Similarity = 0.7890218403032693
-----------------------------------------------------------------------
Lille vs Saint-Étienne
Distance = 2.9665549